In [1]:
import pandas as pd
import numpy as np
from rouge import Rouge 

In [5]:
# dataframe from article retrieval
df = pd.read_csv('question answer.csv',encoding='latin1')
df.drop(['Colonne1','Colonne2'], axis=1, inplace=True)
df.dropna(inplace=True)
df.rename(columns={'answer':'gold_label'}, inplace=True)
print(f'shape: {df.shape}')
display(df.head())

shape: (100, 4)


,id,clean,question,gold_label
0,17307,PARIS When the Islamic State was about to be ...,How old is Mr. Ubelmann ?,36
1,17292,Angels are everywhere in the Mu'iz family's ap...,How much was rent in August ?,$770
2,17298,Finally. The Second Avenue subway opened in Ne...,At which station is there art ?,72nd Street stop
3,17311,WASHINGTON It's or time for Republicans. Afte...,Who is the contemporary Republican Party again...,Mr. Obama
4,17339,"For Megyn Kelly, the shift from Fox News to NB...",How long did Ms. Kelly work for Fox ?,12 years


# Load the Model

In [6]:
from transformers import pipeline
# Replace this with your own checkpoint

model_checkpoint = "huggingface-course/bert-finetuned-squad"
question_answerer = pipeline("question-answering", model=model_checkpoint)

/Users/robinviltoriano/anaconda3/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
snippet = []

for idx, row in df.iterrows():
    
    snippet.append(question_answerer(question=row['question'], context=row['clean']))
    
score = []
answer = []
start_idx = []
end_idx = []
for i in snippet:
    score.append(i['score'])
    answer.append(i['answer'])
    start_idx.append(i['start'])
    end_idx.append(i['end'])

In [8]:
df['snippet'] = snippet
df['score'] = score
df['answer'] = answer
df['start_idx'] = start_idx
df['end_idx'] = end_idx

In [10]:
df.head()

,id,clean,question,gold_label,snippet,score,answer,start_idx,end_idx
0,17307,PARIS When the Islamic State was about to be ...,How old is Mr. Ubelmann ?,36,"{'score': 0.9993014335632324, 'start': 225, 'e...",0.999301,36,225,227
1,17292,Angels are everywhere in the Mu'iz family's ap...,How much was rent in August ?,$770,"{'score': 0.4023560583591461, 'start': 3160, '...",0.402356,$770 a month,3160,3172
2,17298,Finally. The Second Avenue subway opened in Ne...,At which station is there art ?,72nd Street stop,"{'score': 0.621070921421051, 'start': 834, 'en...",0.621071,96th Street,834,845
3,17311,WASHINGTON It's or time for Republicans. Afte...,Who is the contemporary Republican Party again...,Mr. Obama,"{'score': 0.9226491451263428, 'start': 2632, '...",0.922649,Mr. Obama,2632,2641
4,17339,"For Megyn Kelly, the shift from Fox News to NB...",How long did Ms. Kelly work for Fox ?,12 years,"{'score': 0.5684108138084412, 'start': 3214, '...",0.568411,12 years,3214,3222


snippet from article chunk

# Evaluation

In [11]:
import re

def clean_text(text):
    cleaned_text = str(text)
    cleaned_text = cleaned_text.lower()
    cleaned_text = re.sub(r'[^\w\s]', ' ', cleaned_text)
    cleaned_text = re.sub(r'\s+',' ', cleaned_text)
    return cleaned_text.strip()


In [12]:
df['answer_cleaned'] = df.answer.apply(lambda x: clean_text(x))
df['gold_label_cleaned'] = df.gold_label.apply(lambda x: clean_text(x))

In [13]:
df.head()

,id,clean,question,gold_label,snippet,score,answer,start_idx,end_idx,answer_cleaned,gold_label_cleaned
0,17307,PARIS When the Islamic State was about to be ...,How old is Mr. Ubelmann ?,36,"{'score': 0.9993014335632324, 'start': 225, 'e...",0.999301,36,225,227,36,36
1,17292,Angels are everywhere in the Mu'iz family's ap...,How much was rent in August ?,$770,"{'score': 0.4023560583591461, 'start': 3160, '...",0.402356,$770 a month,3160,3172,770 a month,770
2,17298,Finally. The Second Avenue subway opened in Ne...,At which station is there art ?,72nd Street stop,"{'score': 0.621070921421051, 'start': 834, 'en...",0.621071,96th Street,834,845,96th street,72nd street stop
3,17311,WASHINGTON It's or time for Republicans. Afte...,Who is the contemporary Republican Party again...,Mr. Obama,"{'score': 0.9226491451263428, 'start': 2632, '...",0.922649,Mr. Obama,2632,2641,mr obama,mr obama
4,17339,"For Megyn Kelly, the shift from Fox News to NB...",How long did Ms. Kelly work for Fox ?,12 years,"{'score': 0.5684108138084412, 'start': 3214, '...",0.568411,12 years,3214,3222,12 years,12 years


In [22]:
rouge_func = Rouge()
rouge_1_precision = []
rouge_1_recall = []
rouge_1_f1 = []

for idx, row in df.iterrows():
    rouge_score = rouge_func.get_scores(row['answer_cleaned'], row['gold_label_cleaned'])
    rouge_1_precision.append(rouge_score[0]['rouge-1']['p'])
    rouge_1_recall.append(rouge_score[0]['rouge-1']['r'])
    rouge_1_f1.append(rouge_score[0]['rouge-1']['f'])
    
df['rouge_1_precision'] = rouge_1_precision
df['rouge_1_recall'] = rouge_1_recall
df['rouge_1_f1'] = rouge_1_f1

In [25]:
df.head()

,id,clean,question,gold_label,snippet,score,answer,start_idx,end_idx,answer_cleaned,gold_label_cleaned,rouge_1_precision,rouge_1_recall,rouge_1_f1
0,17307,PARIS When the Islamic State was about to be ...,How old is Mr. Ubelmann ?,36,"{'score': 0.9993014335632324, 'start': 225, 'e...",0.999301,36,225,227,36,36,1.000000,1.000000,1.0
1,17292,Angels are everywhere in the Mu'iz family's ap...,How much was rent in August ?,$770,"{'score': 0.4023560583591461, 'start': 3160, '...",0.402356,$770 a month,3160,3172,770 a month,770,0.333333,1.000000,0.5
2,17298,Finally. The Second Avenue subway opened in Ne...,At which station is there art ?,72nd Street stop,"{'score': 0.621070921421051, 'start': 834, 'en...",0.621071,96th Street,834,845,96th street,72nd street stop,0.500000,0.333333,0.4
3,17311,WASHINGTON It's or time for Republicans. Afte...,Who is the contemporary Republican Party again...,Mr. Obama,"{'score': 0.9226491451263428, 'start': 2632, '...",0.922649,Mr. Obama,2632,2641,mr obama,mr obama,1.000000,1.000000,1.0
4,17339,"For Megyn Kelly, the shift from Fox News to NB...",How long did Ms. Kelly work for Fox ?,12 years,"{'score': 0.5684108138084412, 'start': 3214, '...",0.568411,12 years,3214,3222,12 years,12 years,1.000000,1.000000,1.0


In [33]:
pd.DataFrame(df[['rouge_1_precision','rouge_1_recall','rouge_1_f1']].mean()).T

,rouge_1_precision,rouge_1_recall,rouge_1_f1
0,0.783667,0.808929,0.778686
